<a href="https://colab.research.google.com/github/Simurgh818/BMED_6517_emotional_state_classifier/blob/main/FeatureExtractionAndSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotional State Classifier on a multimodal physiological Dataset, [DEAP](https://www.eecs.qmul.ac.uk/mmv/datasets/deap/index.html).

### 1: Download git repo, dataset, and installations.

In [14]:
#importing our git repo
import os
if not os.path.exists('/content/BMED_6517_emotional_state_classifier'):
  !wget https://github.com/Simurgh818/BMED_6517_emotional_state_classifier/blob/main/requirements.txt -q --show-progress --progress=dot
  !git clone https://github.com/Simurgh818/BMED_6517_emotional_state_classifier.git


In [18]:
# Importing the preprocessed dataset files
!wget https://drive.google.com/drive/folders/1_9n-kRKkpnCC2wVovOJsIXOyY3BeW1TB?usp=sharing

--2022-11-13 14:57:41--  https://drive.google.com/drive/folders/1_9n-kRKkpnCC2wVovOJsIXOyY3BeW1TB?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.101, 142.251.2.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘1_9n-kRKkpnCC2wVovOJsIXOyY3BeW1TB?usp=sharing.1’

1_9n-kRKkpnCC2wVovO     [ <=>                ] 219.38K  1.33MB/s    in 0.2s    

2022-11-13 14:57:42 (1.33 MB/s) - ‘1_9n-kRKkpnCC2wVovOJsIXOyY3BeW1TB?usp=sharing.1’ saved [224642]



In [3]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 8.0 MB/s 


In [4]:
import os
import time
import pickle
import pandas as pd
import numpy as np
import pywt

from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps
from scipy.stats import f_oneway

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

from sklearn.decomposition import FastICA

import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from mne.time_frequency import psd_welch
from mne.decoding import cross_val_multiscore

#from fooof import FOOOFGroup
#from fooof.bands import Bands
#from fooof.analysis import get_band_peak_fg
#from fooof.plts.spectra import plot_spectrum

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline

In [5]:
from google.colab import drive
# drive.mount("/content/drive")

KeyboardInterrupt: ignored

#### 1.1: Load data and labels. 

In [6]:
root_folder = '/content/BMED_6517_emotional_state_classifier/'


npy_folder = os.path.join(os.getcwd(), 'results', 'npy')
feats_folder = os.path.join(os.getcwd(), 'results', 'EEG_features')


if not os.path.exists(feats_folder):
    os.makedirs(feats_folder, exist_ok=True)

In [7]:
# Function to load data from each participant file
def read_eeg_signal_from_file(filename):
    x = pickle._Unpickler(open(filename, 'rb'))
    x.encoding = 'latin1'
    p = x.load()
    return p

In [8]:
# Load only 22/32 participants with frontal videos recorded
files = []
for n in range(1, 23): 
    s = ''
    if n < 10:
        s += '0'
    s += str(n)
    files.append(s)
print(files)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


In [16]:
# 22x40 = 880 trials for 22 participants
labels = []
data = []

for i in files: 
  filename = "/content/DEAP/data_preprocessed_python/s" + i + ".dat"
  trial = read_eeg_signal_from_file(filename)
  labels.append(trial['labels'])
  data.append(trial['data'])
   
    
# Re-shape arrays into desired shapes
labels = np.array(labels)
print("Labels - previous: ", labels.shape) # trial x label
labels = labels.flatten()
labels = labels.reshape(880, 4)

data = np.array(data)
print("Data - previous: ", data.shape) # trial x channel x data   
data = data.flatten()
data = data.reshape(880, 40, 8064)

FileNotFoundError: ignored

In [ ]:
# Double-check the new arrays
print("Labels: ", labels.shape) # trial x label
print("Data: ", data.shape) # trial x channel x data

In [ ]:
classes = np.zeros(len(labels))
for i in range(len(labels)):
    if (labels[i,0] < 5) and (labels[i,1] < 5):
        classes[i] = 0 # LVLA
    elif (labels[i,0] > 5) and (labels[i,1] < 5):
        classes[i] = 1 # HVLA
    elif (labels[i,0] < 5) and (labels[i,1] > 5):
        classes[i] = 2 # LVHA
    elif (labels[i,0] > 5) and (labels[i,1] > 5):
        classes[i] = 3 # HVHA                

In [ ]:
# Only extract Valence and Arousal ratings
df_label_classes = pd.DataFrame({'Classes': classes})
df_label_ratings = pd.DataFrame({'Valence': labels[:,0], 'Arousal': labels[:,1], 'Classes': classes})
print(df_label_ratings.describe())

#### 2: Feature Extractions - HR, GSR, Respiration, EOG, EMG, and EEG.